In [21]:
from scipy.special import erf
from scipy.signal import savgol_filter
from matplotlib import pyplot as plt
import numpy as np
from glob import glob
import tifffile as tf
import pandas as pd
from scipy.optimize import curve_fit

In [47]:
t = np.arange(-100*60,100*60,1)

# parameters

D1 = 4e-10 #m^2s^-1
D2 = 1e-9 #m^2s^-1

Q = 20e-9/3600 #m^3s-1
A = (30e-6)*(800e-6)
R_tube = 1.5e-3 #m
L_tube = 0.15 #m
V = np.pi*R_tube**2*L_tube

def erf_input(D,k,t_c):
    return (Q*(t-t_c)/A)/(np.sqrt(4*np.pi*k*D*V/Q))
z1 = erf_input(D1,0,1)
z2 = erf_input(D2,0,1)
y1 = 0.5*(1+erf(z1))
y2 = 0.5*(1+erf(z2))

def background(t,k,t_c):
    D = 4e-10 #m^2s^-1
    z = erf_input(D,k,t_c)
    
    return_val = 0.5*(1+erf(z))
    
    return return_val

/home/marcus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/home/marcus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


In [33]:
0.5*(1+erf(erf_input(1e-10,k = 1,t_c = 0)))

array([0., 0., 0., ..., 1., 1., 1.])

In [34]:
%matplotlib qt
fig, ax = plt.subplots(1,1)

ax.plot(t,y1)
ax.plot(t,y2)

In [8]:
def smooth_decay(I,t0,t1=-1,smoothing_0 = 17,smoothing_1 = 51):
    
    
    if not  smoothing_0 == 0:
        
        if smoothing_0 > t1-t0 and t1 != -1:
            raise ValueError('Smoothing parameter cannot be greater than input data')

        smoothed_decay_0 = I.iloc[t0:t1]

        smoothed_decay_0 = savgol_filter(smoothed_decay_0,smoothing_0,1)
    else:
        smoothed_decay_0 = I.iloc[t0:t1]
    
    if t1 != -1:
        
        smoothed_decay_1 = I.iloc[t1:]
        smoothed_decay_1  = savgol_filter(smoothed_decay_1,smoothing_1,1)
        
        smoothed_decay = np.concatenate((smoothed_decay_0,smoothed_decay_1))
    else:
        smoothed_decay = smoothed_decay_0
   
    if not smoothing_0 == 0:
        final_smoothed_decay = savgol_filter(smoothed_decay,smoothing_0,1)
    else:
        final_smoothed_decay = smoothed_decay
    return final_smoothed_decay
                                 

In [37]:
tifs = glob('/media/marcus/9C33-6BBD/20210427_OmpF_PCPG_KCL_Flush_2-20210427T184510Z-002/20210427_OmpF_PCPG_KCL_Flush_2/20210427_OmpF_PCPG_KCL_Flush_2_MMStack_Pos0.ome.tif')
tif = tifs[-1]

dt = 9/60

vid = tf.TiffFile(tif)

frames = vid.asarray()
bg = np.median(frames[:,447:512,396:440],axis = (1,2))
# bg = bg[:
bg = np.array(bg)-np.mean(bg[-20:-10])
bg = bg/np.max(bg)
bg_k = 1-bg
# make pandas dataframe 

bg= pd.DataFrame({'Bg': bg})
smoothed_bg = smooth_decay(bg['Bg'],0,150,smoothing_0=7,smoothing_1=11)


OME series: invalid TiffData index
OME series: invalid TiffData index
OME series: invalid TiffData index
OME series: invalid TiffData index


# Fit erf to background intensity


In [48]:

# estimate the offset time by the 0.5 point time
# the relative diffusion to ideal diffusion constant needs to be estimated with experience

p0 = [0.5,34]

t = dt*np.arange(bg_k.shape[0])

params = curve_fit(background,t,bg_k,p0)



/home/marcus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in sqrt
  from ipykernel import kernelapp as app


In [49]:
bg_bar = background(t,2*params[0][0],params[0][1])

In [50]:
fig, ax = plt.subplots(1,1)
ax.plot(t,bg_bar)
ax.plot(t,bg_k)